In [10]:
from importlib.metadata import version
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
import numpy as np
import math
from dataclasses import dataclass
from matplotlib import pyplot as plt
import time
import os

# Pour torch si vous avez un GPU
# device = "cpu" if not torch.cuda.is_available() else "cuda"
device = "cpu" # Pour forcer l'utilisation du CPU

In [11]:
# Environement potentielement testé
from environnement.environnement import Environnement as env # mother class
from environnement.environnement1 import Environnement1 as env1
from environnement.environnement2Str import Environnement2 as env2Str
from environnement.environnement3Str import Environnement3 as env3Str
from environnement.environnement6Str import Environnement6 as env6Str
from environnement.small_loop import small_loop

# model machine learning
from model.DeepNN import *
from model.Tokenizer import *
from model.CustomLoader import CustomLoader
from outil import *
from inter.interactions import Interaction
from inter.simpleInteraction import simpleInteraction as inter

# L'agent qui voit les paterns
Nos précedants agent fonctionnent pour des environements simple, dont le feedback ne dépand que de l'action choisit. Nous aimerions maintenant un agent capable de prédrie un feedback dans des environement dans lequel l'ouctome dépand des précédantes interactions. 

## Mécanisme de prédiction
L'agent en lui même n'est pas différent de l'agent 2. Le changement a apporté se situe au niveau de l'entrainement du modèl. A la plade de lui passer seulement une action, nous allons lui donner une ou plusieurs interaction, suivit d'une action.

### Exemples: 
Imaginons que notre agent a fais cette séquence d'interaction :

- a, b : actions
- x, y : feedback
> a, x, b, y

- Nous allons passer au modèls a, x, b
- Et nous allons lui apprendre a prédire b

L'idée derière est de reconnaitre des patterns, imaginons cette séquence d'interaction :
> a, x, b, y, b, x, a, y, a, x

Ici notre environement renvoie 'y' seulement quand l'action précédente est différente de celle actuelle. Nous allons passer comme jeu d'apprentissage au modèls :

- a, x, b => y
- b, y, b => x
- b, x, a => y
- a, y, a => x

Le but est d'entrainer le modèls à reconnaitre des patterns pour prédire le bon feedback.

## Exploration
Pour pouvoir prédire corectement en prennant en commpte les patterns, il faut que l'agent est put les voirs au moins une fois. Pour se faire nous devons modifier notre fonction d'exploration pour tester toutes les combinaisons possible.

In [ ]:
class Agent4:
    def __init__(self, model, all_outcomes, all_actions, valance, tokenizer, optimizer=None, loss_func=None):
        """ 
        Création de l'agent.
        
        - self._action : action précédente
        - self._predicted_outcome : prédiction de l'outcome précédent
        """
        self._action = None
        self._predicted_outcome = None
        self._model = model
        self._otimizer = optimizer
        self._loss_func = loss_func
        self._tokenizer:SimpleTokenizerV1 = tokenizer
        self._all_outcomes = all_outcomes
        self._all_actions = all_actions
        self._history_act = []
        self._history_fb = []
        self._valance=valance

    # Cette fonction est différente de l'agent 2
    def fit(self, actions:list, outcomes:list, nb_epoch:int= 5, validate_loader=None):
        """
        Fonction d'entrainement de l'agent
        """
        context_lenght = self._model.input_size
        print(f'context_lenght {context_lenght}')
        if len(actions) + len(outcomes) < context_lenght:
            raise Exception("Not enough data to train model")

        actions = [[self._tokenizer.encode(act)] for act in actions]
        outcomes = self._tokenizer.encode(outcomes)
        
        if isinstance(self._model, torch.nn.Module):
            self._model.train()
            actions = torch.tensor(actions, dtype=torch.float).to(device) # On passe toutes les actions que l'agent a fais
            outcomes = torch.tensor(outcomes, dtype=torch.long).to(device) # On passe toutes oputcomes qu'il a recut
            data_loarder = CustomLoader(actions=actions, outcomes=outcomes, # On va créer un dataset
                         context_lenght= context_lenght)
            
            # A la place de donnée x = [act1] y = [out1]
            # Nous voulons donné : x = [act1, out1, act2] y = [out2]

            data_loader = torch.utils.data.DataLoader( # On utilise torch pour charger les données
                data_loarder,batch_size=32, shuffle=True)

            train_with_batch(model=self._model, 
                    train_loader=data_loader,
                    optimizer=self._otimizer,
                    loss_func=self._loss_func,
                    nb_epochs=nb_epoch,
                    validate_loader=validate_loader,
                    print_=True)
        else: # Si le model n'est pas un model pytorch
            raise Exception('Not implemented')
            self._model.fit(action, outcome)
            pass

    def get_prediction(self, action):
        gap = (self._model.input_size - 1) // 2
        x = []
        for i in range(len(self._history_act) - gap, len(self._history_act)):
            print(self._history_act)
            x.append(self._history_act[i])
            x.append(self._history_fb[i])
        x.append(action)
        action = self._tokenizer.encode(x)
        
        if isinstance(self._model, torch.nn.Module):
            self._model.eval() 
            action = torch.tensor(action, dtype=torch.float).to(device)
            x = self._model(action)
            x = torch.nn.functional.softmax(x, dim=0)
        else:
            raise Exception('Not implemented')
            x=self._model.predict(action)
        
        return x
    
    def check_all_actions(self):
        act_to_test = None
        for act in self._all_actions:
            if act not in self._history_act:
                act_to_test = act
                break
        return act_to_test
    
    def decide(self):
        """
        Fonction qui choisit l'action a faire en fonction des prédictions \
        du modèles entrainné. Nous renforçons choisisons les actions que \
        ou le modèle n'est pas sûr.
        """

        act_test = self.check_all_actions()
        if act_test:
            print("i don't know", act_test)
            self._action = act_test
            return act_test

        best_act = self._all_actions[0]
        best_expected_val = -np.inf

        # On vérifie que l'on a vue assez d'interaction pour faire des prédictions
        if len(self._history_act) + len(self._history_fb) < self._model.input_size:
            print("Not enough data to make a decision")
            return best_act

        # Vérifie si le modèles est sur de sa prédiction
        for act in self._all_actions:
            probs:torch.Tensor = self.get_prediction(act)
            max_prob = torch.max(probs).item()
            # Formule utiliser : 1 / n + 0.5 / n
            print(f'for action {act} probs {probs} max_prob {max_prob}')
            if max_prob < 1 / probs.size(dim=0) + 0.5 / probs.size(dim=0):
                print("je ne suis pas sur de ", act)
                if len(self._all_actions) + len(self._all_outcomes) > self._model.input_size:
                    self.fit(self._history_act, self._history_fb, validate_loader=None)

            probs:torch.Tensor = self.get_prediction(act)
            if max_prob < 1 / probs.size(dim=0) + 0.5 / probs.size(dim=0):
                print("je n'arrive pas à être sur de ", act)
                return act
            
            # Si le modèle as une prédiction sur, on regarde sa valance
            predi = self._tokenizer.decode(torch.argmax(probs, dim=0).item())
            expected_val = self._valance[inter(act, predi)]
            if expected_val > best_expected_val:
                best_act = act
                best_expected_val = expected_val
                print(f"Action: {act}, Expected valance: {expected_val}")
        self._action = best_act
        return best_act

    # Modifier
    def predict(self, action):
        """
        Funciton de prédiction
        """

        if len(self._history_act) + len(self._history_fb) + 1 < self._model.input_size:
            raise Exception("Not enough data to train model")
        
        gap = (self._model.input_size - 1) // 2
        x = []
        for i in range(len(self._history_act) - gap, len(self._history_act)):
            print(self._history_act)
            x.append(self._history_act[i])
            x.append(self._history_fb[i])
        x.append(action)
        action = self._tokenizer.encode(x)
        if isinstance(self._model, torch.nn.Module):
            self._model.eval() 
            action = torch.tensor(action, dtype=torch.float).to(device)
            x = self._model(action)
            x = torch.argmax(x, dim=0).item()

        else:
            raise Exception('Not implemented')
            x=self._model.predict(action)
        
        return self._tokenizer.decode(x)

    # Modofier
    def action(self, outcome, fit=True, validate_loader=None):
        """ 
        Fonction qui choisit l'action a faire en fonction de la dernière \
        intéraction avec l'environnement. \n
        C'est ici que nous allons implémenter un mécanisme de ML \
        pour choisir la prochaine action.

        :param: **outcome** feedback de la dernière intéraction avec l'environnement

        :return: **action** action à effectuer
        """
        if self._action is not None:
            self._history_fb.append(outcome)
            print(f"Action: {self._action}, Prediction: {self._predicted_outcome}, Outcome: {outcome}, " 
                  f"\033[0;31m Satisfaction: {self._predicted_outcome == outcome} \033[0m")
            if self._predicted_outcome != outcome:
                if len(self._history_act) + len(self._history_fb) > self._model.input_size:
                    self.fit(self._history_act, self._history_fb, validate_loader=validate_loader)
                else:
                    self._action = self._all_actions[0]

            # Maintenant nous choisissons la prochaine action en fonction de la valance
            self._action = self.decide()
            if len(self._history_act) + len(self._history_fb) + 1 > self._model.input_size:
                self._predicted_outcome = self.predict(self._action)
            self._predicted_outcome = self.predict(self._action)
            self._history_act.append(self._action)
        else:
            self._action = self._all_actions[0]
            self._history_act.append(self._action)
            print(f'len all interaction {len(self._history_act) + len(self._history_fb)}')
            print(f'input size {self._model.input_size}')
            
            print(f"Action de base : {self._action} Prediction: {self._predicted_outcome}")
        
        return self._action, self._predicted_outcome

In [ ]:
env_test2 = env3Str()

model_ML = DeepNetwork(hidden_size=[10, 5], input_size=3, output_size=2)
optimizer = torch.optim.Adam(model_ML.parameters(), lr=1e-1, weight_decay=1e-2)
loss_func = nn.CrossEntropyLoss()
tokenizer = SimpleTokenizerV1(create_dico_numerate_word(env_test2.get_outcomes() + env_test2.get_actions()))

valence = {
    inter('a', 'x') : -1,
    inter('a', 'y') : 1,
    inter('b', 'x') : -1,
    inter('b', 'y') : 1
}
agent_test2 = Agent4(
    model=model_ML,
    all_outcomes= env_test2.get_outcomes(),
    all_actions= env_test2.get_actions(),
    valance=valence,
    tokenizer=tokenizer,
    optimizer=optimizer,
    loss_func=loss_func)

history_good = []
pourcent_by_10 = []
outcome = None
for i in range(25):
    print(f"=======================\033[0;32m iteration {i} \033[0m=======================")
    action, predi = agent_test2.action(outcome, False)
    outcome = env_test2.outcome(action)
    history_good.append(outcome == predi)
    pourcent_by_10.append(sum(history_good[-10:]) * 10 if len(history_good) >= 10 else 0)
    print(f'action {action} predi {predi} outcome {outcome}')
    print(f"Action choisie : {action} \033[0;34m{pourcent_by_10[-1]} \033[0m")
    print("\n")
    

liste hidden init [10, 5]
======================= iteration 0 =======================
len all interaction 1
input size 3
Action de base : a Prediction: None
action a predi None outcome x
Action choisie : a 0 


======================= iteration 1 =======================
Action: a, Prediction: None, Outcome: x,  Satisfaction: False 
i don't know b
['a']
action b predi x outcome y
Action choisie : b 0 


======================= iteration 2 =======================
Action: b, Prediction: x, Outcome: y,  Satisfaction: False 
context_lenght 3
for this action tensor([[2.],
        [3.]]) and this outcom tensor([0, 1]) and this context 3
gap is 2 and result is -1
for this action tensor([[2.],
        [3.]]) and this outcom tensor([0, 1]) and this context 3
gap is 2 and result is -1
for this action tensor([[2.],
        [3.]]) and this outcom tensor([0, 1]) and this context 3
gap is 2 and result is -1
for this action tensor([[2.],
        [3.]]) and this outcom tensor([0, 1]) and this context 3